In [21]:
import numpy as np
import pandas as pd

class Node:
# for every node we ask a question,which have 2 answers true and false. split_col and split_value represents the question to ask
#on which column and what value.(example- petal width<=0.8)and true false represents the direction to go if answer is true or false 
    def __init__(self,split_col,split_val,output):
        self.split_col = split_col
        self.split_val = split_val
        self.output = output
        self.true = None
        self.false = None

class ShoddyDTClassifier:
    def __init__(self):
        self.__root = None #root represents root of tree after fitting
        
    def __is_pure(self, y):
    # checks if node is pure
        unique_classes = np.unique(y)
        if len(unique_classes) == 1:
            return True
        else:
            return False
        
    def __classify_data(self, y):
    # in case node is pure it returns the only class in the node,otherwise returns class with majority
        unique_classes,count = np.unique(y,return_counts=True) #returns 2 arrays, 1st with unique classes and 2nd with count of each classes
        max_count_index = np.argmax(count) #argmax function gives index of largest number in array
        classification = unique_classes[max_count_index]
        return classification
    
    def __get_possible_splits(self,x):
    # function returns dictionary of all the midpoints of continuous data with column index as keys and midpoints as values
        possible_splits={}
        for column_index in range(x.shape[1]):
            possible_splits[column_index] = [] #creating list of values for each column index as keys
            values = x[:,column_index]
            unique_values = np.unique(values)
            for i in range(len(unique_values)-1):
                splits = (unique_values[i]+unique_values[i+1])/2 #getting midpoints of every 2 consecutive values
                possible_splits[column_index].append(splits)
        return possible_splits
    
    def __split_data(self,x,y,selected_feature,split_value):
    #function returns 2 arrays one with target values below our split value one with above our split value
    # example- data_below = all target values below 0.8, data_above = all target values above 0.8
        data_below = y[x[:,selected_feature] <= split_value]
        data_above = y[x[:,selected_feature] > split_value]
        return data_below,data_above
    
    def __get_entropy(self,col):
    #counts entropy
        val_arr,count=np.unique(col,return_counts=True)
        p=count/count.sum()
        p_log=p*np.log2(p)
        entropy= -p_log.sum()
        return entropy
    
    def __get_overall_entropy(self,data_below,data_above):
    # this function is used to determine the overall weighted entropy of data below and data above 
        total_data_points = len(data_below) + len(data_above)
        p_data_below = len(data_below) / total_data_points
        p_data_above = len(data_above) / total_data_points
        overall_entropy = (p_data_below * self.__get_entropy(data_below)) + (p_data_above * self.__get_entropy(data_above))
        return overall_entropy
    
    def __get_best_split(self,x,y,possible_splits):
    # this function is used to get best split value(ex. 0.8) and best column index by calculating overall entropy
        overall_entropy = np.inf
        for col_index in possible_splits:
            for values in possible_splits[col_index]:
                data_below,data_above = self.__split_data(x,y,col_index,values)
                curr_overall_entropy = self.__get_overall_entropy(data_below,data_above)
                if curr_overall_entropy <= overall_entropy:
                    overall_entropy = curr_overall_entropy
                    best_split_column = col_index
                    best_split_value = values
        return best_split_column,best_split_value
    
    def __gain_ratio(self, selected_feature_index, split_value,x, y):
    #calculates gain ratio for every split
        original_entropy = self.__get_entropy(y)
        initial_size = y.shape[0]
        info_f = 0
        split_info = 0
        D1 = y[x[:,selected_feature_index] <= split_value]
        D2 = y[x[:,selected_feature_index] > split_value]
        info_f = (len(D1) / initial_size) * self.__get_entropy(D1) + (len(D2) / initial_size) * self.__get_entropy(D2)
        split_info = -((len(D1) / initial_size) * np.log2(len(D1) / initial_size) + (len(D1) / initial_size) * np.log2(len(D1) / initial_size))
        if split_info==0:
            return np.inf
        info_gain = original_entropy - info_f
        gain_ratio=info_gain/split_info
        return gain_ratio
    
    def __build_decision_tree(self,x,y,level,splits):
        
        # base case- if data is pure,classify the data and return a node with only output
        if self.__is_pure(y):
            classification = self.__classify_data(y)
            print('Level',level)
            for i in set(y):
                print('Count of',i,' = ',len(y))
                print('Current Entropy is = 0.0',)
                print('Reached leaf node')
            return Node(None,None,classification)
        
        #getting best feature to split upon
        best_feature,best_split_value = self.__get_best_split(x,y,splits)
        
        # printing job
        print('Level',level)
        unique_classes,count = np.unique(y,return_counts=True)
        for i in range(len(unique_classes)):
            print('Count of',unique_classes[i],'=',count[i])
        print('Current Entropy is =',self.__get_entropy(y))
        print('Splitting on feature X'+str(best_feature),'with gain ratio',self.__gain_ratio(best_feature,best_split_value,x,y))
        
        #recursive part
        classification = self.__classify_data(y) #classify data to put in node output
        node = Node(best_feature,best_split_value,classification) #creating node
        x_data_below = x[x[:,best_feature]<=best_split_value] # all the x data below split value(ex. 0.8)
        x_data_above = x[x[:,best_feature]>best_split_value] # all the x data above split value
        y_data_below = y[x[:,best_feature]<=best_split_value] #all the target data above split value
        y_data_above = y[x[:,best_feature]>best_split_value] #all the target data above split value
        true = self.__build_decision_tree(x_data_below,y_data_below,level+1,splits) #gets true answer
        false = self.__build_decision_tree(x_data_above,y_data_above,level+1,splits) #gets false answer
        node.true = true
        node.false = false
        return node #returning root of tree
    
    def fit(self,x,y):
        splits = self.__get_possible_splits(x)
        self.__root = self.__build_decision_tree(x,y,0,splits)
        
    def __predict_helper(self,x,root):
    #starts from root node and asks a question to find out which direction to go    
        
        #base case- if root has no child,return the output
        if root.true == None and root.false == None:
            return root.output
        
        #recursive part
        if x[root.split_col] <= root.split_val: # if data is less than specified value we go towards true
            return self.__predict_helper(x,root.true)
        else:
            return self.__predict_helper(x,root.false) #otherwise we go towards false
        
    def predict(self,x):
        y = np.array([0 for i in range(len(x))])
        for row in range(len(x)):
            y[row] = self.__predict_helper(x[row],self.__root)
        return y
    
    def score(self,x,y):
    #returns mean accuracy
        y_pred = self.predict(x)
        count = 0
        for i in range(len(y)):
            if y_pred[i] == y[i]:
                count+=1
        return count/len(y)

#data preparation
from sklearn.datasets import load_iris
iris=load_iris()
x = iris.data
y = iris.target
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y) 
clf=DecisionTreeClassifier()
s_clf=ShoddyDTClassifier()
clf.fit(x_train,y_train)
s_clf.fit(x_train,y_train)
c=clf.score(x_test,y_test)
s=s_clf.score(x_test,y_test)
print('----------------------------')
print('sklearn score=',c)
print('our score=',s)


Level 0
Count of 0 = 39
Count of 1 = 37
Count of 2 = 36
Current Entropy is = 1.5841606263806214
Splitting on feature X3 with gain ratio 0.8797391878691854
Level 1
Count of 0  =  39
Current Entropy is = 0.0
Reached leaf node
Level 1
Count of 1 = 37
Count of 2 = 36
Current Entropy is = 0.9998646331239298
Splitting on feature X3 with gain ratio 0.8238088893085824
Level 2
Count of 1 = 36
Count of 2 = 1
Current Entropy is = 0.1792560669283215
Splitting on feature X2 with gain ratio 2.3304202679737513
Level 3
Count of 1  =  36
Current Entropy is = 0.0
Reached leaf node
Level 3
Count of 2  =  1
Current Entropy is = 0.0
Reached leaf node
Level 2
Count of 1 = 1
Count of 2 = 35
Current Entropy is = 0.18312206830137276
Splitting on feature X2 with gain ratio 0.2753283606576246
Level 3
Count of 1 = 1
Count of 2 = 1
Current Entropy is = 1.0
Splitting on feature X1 with gain ratio 1.0
Level 4
Count of 2  =  1
Current Entropy is = 0.0
Reached leaf node
Level 4
Count of 1  =  1
Current Entropy is = 0.